In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [77]:
# spectral clustering
def laplacian(A): # form the Laplacian
    n = A.shape[0]
    D = sum(A.transpose())
    D = np.diag(np.array(D))
    Laplacian = np.eye(n) - np.dot(np.dot(np.linalg.inv(np.sqrt(D)),A),np.linalg.inv(np.sqrt(D)))
    return Laplacian
def top_k_eigenvectors(L,k): #return top k evectors of laplacian
    eig, V = np.linalg.eig(L)
    return V[:,0:k]
def normalize_pts(V): # 
    V_n = normalize(V,norm='l2',axis=1)
    return V_n
def spec_clustering(A,k):
    L = laplacian(A)
    V = top_k_eigenvectors(L,k)
    V_n = normalize_pts(V)
    return V_n

In [78]:
# Playing with Graphs and graph clustering
# Here is a random graph on 30 nodes with 200 edges selected at random
G = nx.gnm_random_graph(30,200)
nx.draw(G)
plt.show()

In [ ]:
# generate similarity matrix exp{-1/2 ||x_i - x_j||^2}
def gen_similarity(X): # points entered as columns
    n = X.shape[1]
    A = 1.0*np.zeros((n,n))
    for i in np.arange(n):
        for j in np.arange(n):
            A[i,j] = np.linalg.norm(X[:,i] - X[:,j])
    A = -0.5*(A**2)/2
    A = np.exp(A)
    A = sparsify_eps(A,0.05)
    return A
def sparsify_eps(A,eps):
    A = A*1.0*(A > eps)   
    return A

In [2]:
# Graph Similarity: create a (p,q) balanced Stoch Block Model adjacency matrix
def create_adjacency(p,q,n):
    A11 = np.random.rand(n/2,n/2)
    A11 = 0.5*(A11 + A11.transpose())
    A11 = 1*(A11 < p)
    A12 = np.random.rand(n/2,n/2)
    A12 = 1*(A12 < q)
    A21 = A12.transpose()
    A22 = np.random.rand(n/2,n/2)
    A22 = 0.5*(A22 + A22.transpose())
    A22 = 1*(A22 < p)
    A1 = np.concatenate((A11,A12),axis=1)
    A2 = np.concatenate((A21,A22),axis=1)
    A = np.concatenate((A1,A2))
    return A

In [3]:
# playing with the Stochastic Block Model
n = 300
A = create_adjacency(0.75,0.25,n)

In [5]:
# Plot with rows and columns optimally ordered
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(A,cmap="hot")
plt.colorbar()
plt.show()

In [5]:
# shuffle rows and columns of A
p = np.arange(n)
np.random.shuffle(p)
M1 = A[p,:]
M2 = M1[:,p]

In [6]:
# Plot with the randomly shuffled row/column order
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(M2,cmap="hot")
plt.colorbar()
plt.show()

In [8]:
# Run the spectral clustering algorithm on the shuffled rows/columns
V = spec_clustering(M2,2)

In [9]:
# This is the output of the spectral clustering algorithm
V1 = V[:,0]
V2 = V[:,1]
plt.plot(V1,V2,"o")
plt.show()

In [10]:
# The above suggests a simple way to try to find the optimal ordering
def re_order_rows(A,V2):
    index1 = 1*(V2 > 0)
    order1 = np.where(index1 ==1)[0]
    order2 = np.where(index1 ==0)[0]
    order = np.concatenate((order1,order2))
    AA1 = A[order,:]
    AA2 = AA1[:,order]
    return AA2

In [12]:
# We re-order using the above rule, and plot again
AA = re_order_rows(M2,V2)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(AA,cmap="hot")
plt.colorbar()
plt.show()

In [73]:
# A different clustering problem: Create inner and outer rings
N = 400
theta = np.random.rand(N)*2*np.pi
X_outer = 10*np.concatenate(([np.cos(theta)],[np.sin(theta)]),axis=0)
X_outer = X_outer + np.random.randn(X_outer.shape[0],X_outer.shape[1])*0.1
X_inner = 0.5*np.concatenate(([np.cos(theta)],[np.sin(theta)]),axis=0)
X_inner = X_inner + np.random.randn(X_outer.shape[0],X_outer.shape[1])*0.1


In [69]:
# This is what it looks like
plt.plot(X_outer[0,:],X_outer[1,:],"o")
plt.plot(X_inner[0,:],X_inner[1,:],"x")
plt.show()

In [74]:
# We put all the points together
# Then create a similarity matrix: exp{-0.5*||X_i - X_j||^2}
X = np.concatenate((X_outer,X_inner),axis=1)
A_circ = gen_similarity(X)
V_circ = spec_clustering(A_circ,2)

In [75]:
# Let's see how well this does
V1_circ = V_circ[:,0]
V2_circ = V_circ[:,1]
plt.plot(V1_circ,V2_circ,"o")
plt.show()

In [76]:
# Another way to see how well it does
1*(V1_circ > -0.4)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,